#  주요 언어 모델 공급자 및 활용 

- Gemini, Groq, Ollama, Huggingface

### **학습 목표:** 주요 언어 모델 공급자들의 특징을 비교하고 적절한 활용 방안을 수립한다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

import uuid

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [ ]:
from langfuse.callback import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

# 정상 작동 확인
langfuse_handler.auth_check() 

`(4) 벡터스토어 로드`

In [ ]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

`(5) 백터 검색기 생성`

In [ ]:
# 기본 retriever 초기화
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 4}
)

query = "테슬라의 자율주행 기술의 특징은 무엇인가요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"{doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

`(5) RAG 체인 생성`

In [ ]:
# 각 쿼리에 대한 검색 결과를 한꺼번에 Context로 전달해서 답변을 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def create_rag_chain(retriever, llm):

    template = """Answer the following question based on this context. If the context is not relevant to the question, just answer with '답변에 필요한 근거를 찾지 못했습니다.'

    [Context]
    {context}

    [Question]
    {question}

    [Answer]
    """

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([doc.page_content for doc in docs])

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()} 
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [ ]:
# RAG 체인 생성 및 테스트 (OpenAI의 gpt-4.1-mini 사용)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")
openai_rag_chain = create_rag_chain(chroma_k_retriever, llm)

query = "테슬라의 자율주행 기술의 특징은 무엇인가요?"
answer = openai_rag_chain.invoke(query)

print(answer)

---

## 주요 **LLM 모델 공급자** 

- **LLM 공급자**는 대규모 언어 모델을 개발하고 서비스를 제공하는 기업들을 의미함
- **기술 서비스** 제공 방식에 따라 다양한 기업들이 시장에 참여하고 있음


### 1) **Google Gemini** API

- **Google의 최신 AI 모델** Gemini가 API를 통해 개발자들에게 공개됨
- API는 **세 가지 모델 버전**을 제공: Gemini Pro, Gemini Pro Vision, Ultra 모델
- **텍스트, 이미지, 오디오** 등 다양한 형식의 입력을 처리할 수 있는 **멀티모달 기능** 지원
- 무료 체험판과 함께 **사용량 기반 과금 체계** 도입

- 참고: https://ai.google.dev/
- 설치: `pip install langchain_google_genai` / ```poetry add langchain_google_genai```

- **API 키 설정**
- 환경 변수: **GOOGLE_API_KEY**

In [ ]:
# ChatGoogleGenerativeAI - LLM 모델 
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 로드 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# RAG 체인 생성 및 테스트
google_genai_rag_chain = create_rag_chain(chroma_k_retriever, llm)

query = "테슬라의 자율주행 기술의 특징은 무엇인가요?"
answer = google_genai_rag_chain.invoke(query)

print(answer)

### **[실습]**

- Google Gemini에서 제공하는 다른 모델을 RAG 체인에 적용합니다. 
- 각 모델이 생성한 답변의 품질과 특성을 비교합니다. 
- 모델 확인: https://ai.google.dev/gemini-api/docs

In [ ]:
# 여기에 코드를 작성하세요.

### 2) **Groq** API

- **초고속 추론 속도**를 제공하는 새로운 LLM API 서비스 출시
- **오픈소스 모델** 지원으로 Llama, Mixtral 등 다양한 기존 모델 활용 가능
- **1초 미만의 응답 시간**을 목표로 하는 **LPU**(Language Processing Unit) 기술 도입
- API 사용량에 따른 **합리적인 가격 정책** 제시

- 참고: https://groq.com/
- 설치: `pip install langchain_groq` / ```poetry add langchain_groq```

- **API 키 설정**
- 환경 변수: **GROQ_API_KEY**

In [ ]:
load_dotenv()

In [ ]:
# ChatGroq - LLM 모델 
from langchain_groq import ChatGroq

# 모델 로드 
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.0,
    max_tokens=100,
)

# RAG 체인 생성 및 테스트
groq_rag_chain = create_rag_chain(chroma_k_retriever, llm)

query = "테슬라의 자율주행 기술의 특징은 무엇인가요?"
answer = groq_rag_chain.invoke(query)

print(answer)

### **[실습]**

- Google Gemini에서 제공하는 다른 모델을 RAG 체인에 적용합니다. 
- 각 모델이 생성한 답변의 품질과 특성을 비교합니다. 
- 모델 확인: https://ai.google.dev/gemini-api/docs

In [ ]:
# 여기에 코드를 작성하세요.

### 3) **Ollama** API

- **로컬 환경**에서 LLM을 쉽게 실행할 수 있는 **오픈소스 플랫폼**
- **다양한 LLM 모델** 지원 (Llama, Mistral, CodeLlama 등)
- **단순한 명령어**로 모델 다운로드 및 실행 가능
- 개인용 컴퓨터에서 AI 모델을 쉽게 활용할 수 있는 효율적인 솔루션 제공 (**macOS, Linux, Windows** 운영체제 지원)

- 웹사이트: https://ollama.com/
- 깃허브: https://github.com/ollama/ollama
- 설치: `pip install langchain_ollama` / ```poetry add langchain_ollama```

In [ ]:
# ChatOllama - LLM 모델 
from langchain_ollama import ChatOllama

# 모델 로드 
llm = ChatOllama(
    model = "qwen2.5:latest",   # qwen2.5:latest 모델 적용 (ollama pull qwen2.5:latest)
    temperature = 0,
    num_predict = 200,
)

# RAG 체인 생성 및 테스트
ollama_rag_chain = create_rag_chain(chroma_k_retriever, llm)

query = "테슬라의 자율주행 기술의 특징은 무엇인가요?"
answer = ollama_rag_chain.invoke(query)

print(answer)

### **[실습]**

- Google Gemini에서 제공하는 다른 모델을 RAG 체인에 적용합니다. 
- 각 모델이 생성한 답변의 품질과 특성을 비교합니다. 
- 모델 확인: https://ai.google.dev/gemini-api/docs

In [ ]:
# 여기에 코드를 작성하세요.